In [ ]:
import csv
import math
import os
import pandas as pd
import pathlib
import re

In [ ]:
list_extractor = {
    'lbp': [59],
    'surf64': [128, 256, 257],
    'surf128': [128, 256, 512, 513],
    'mobilenetv2': [128, 256, 512, 1024, 1280],
    'resnet50v2': [128, 256, 512, 1024, 2048],
    'vgg16': [128, 256, 512]
}
list_classifier = ['DecisionTreeClassifier', 'KNeighborsClassifier', 'MLPClassifier', 'RandomForestClassifier', 'SVC']
list_dim = [256, 400, 512]
list_metrics = ['mean', 'std']
list_source = ['manual', 'unet']
list_time = ['search_best_params', 'train_valid']

path = 'out'

In [ ]:
columns = [c + '_' + str(d) + '_' + s for c in list_classifier for s in sorted(list_source) for d in list_dim]
columns = list(filter(lambda x: not('400_manual' in x) and not('512_manual' in x), columns))

index_time = [e + '_' + str(d) + '_' + t for e in list_extractor.keys() for d in reversed(list_extractor[e]) for t in list_time]
index = [e + '_' + str(d) + '_' + m for e in list_extractor.keys() for d in reversed(list_extractor[e]) for m in list_metrics]

In [ ]:
df = pd.DataFrame(columns=columns, index=index)
df_time = pd.DataFrame(columns=columns, index=index_time)

In [ ]:
def file_is_colormode_and_slice_and_patch(file):
    data_info = pd.read_csv(os.path.join(str(file).replace('mean.csv', 'info.csv')), sep=";", header=None, index_col=0)
    color_mode = data_info.loc['color_mode'][1]
    slice = data_info.loc['slice'][1]
    n_patch = data_info.loc['n_patch'][1]
    return True if color_mode.lower() == 'grayscale' and (slice == 'horizontal' or math.isnan(slice)) and (n_patch == '3' or math.isnan(n_patch)) else False

list_files = [p for p in pathlib.Path(path).rglob('mean.csv') if p.is_file()]
list_files = [file for file in list_files if file_is_colormode_and_slice_and_patch(file)]

for file in list_files:
    _, _, source, dim_image, extractor, classifier, _, n_features, _ = re.split('/', str(file))
    data = pd.read_csv(file, sep=";", header=None, index_col=0).squeeze()
    index_mean = extractor + '_' + str(n_features) + '_mean'
    index_std = extractor + '_' + str(n_features) + '_std'
    column = classifier + '_' + dim_image + '_' + source
    df.loc[index_mean, column] = data.loc['mean_sum'][1]
    df.loc[index_std, column] = '±' + str(float(data.loc['std_sum'][1]))

    index_mean_time_train_valid = extractor + '_' + str(n_features) + '_train_valid'
    index_mean_time_search_best_params = extractor + '_' + str(n_features) + '_search_best_params'
    df_time.loc[index_mean_time_train_valid, column] = data.loc['mean_time_train_valid'][1]
    df_time.loc[index_mean_time_search_best_params, column] = data.loc['mean_time_search_best_params'][1]
df

In [ ]:
df.to_csv(os.path.join('mean.csv'), sep=';', na_rep='', quoting=csv.QUOTE_ALL)
df_time.to_csv(os.path.join('mean_time.csv'), sep=';', na_rep='', quoting=csv.QUOTE_ALL)
df_time